In [1]:
import os
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
video_files = glob.glob('video_footages/*')
video_files

['video_footages\\Raw Video_ Pittsburg Neighborhood Drive-By Shootings.mp4',
 'video_footages\\Shooting captured by surveillance camera in Parma.mp4',
 'video_footages\\Shopping, People, Commerce, Mall, Many, Crowd, Walking   Free Stock video footage   YouTube.mp4',
 'video_footages\\snow dispute.mp4',
 'video_footages\\Surveillance camera captures tanks entering Ukraine via Belarus border.mp4',
 'video_footages\\trail cam video creatures of the night.mp4',
 'video_footages\\Unbelievably busy bicycle crossing in Amsterdam.mp4',
 'video_footages\\vecteezy_people-crossing-the-road-on-zebra-tallin_28257759.mp4',
 'video_footages\\차흐르듯 (1).mp4',
 'video_footages\\차흐르듯.mp4']

In [3]:
def get_motion_mask(fg_mask, min_thresh=0, kernel=np.array((9,9), dtype=np.uint8)):
    """ Obtains image mask
        Inputs: 
            fg_mask - foreground mask
            kernel - kernel for Morphological Operations
        Outputs: 
            mask - Thresholded mask for moving pixels
        """
    _, thresh = cv2.threshold(fg_mask,min_thresh,255,cv2.THRESH_BINARY)
    #mask = cv2.adaptiveThreshold(fg_mask, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV, 11, 3)
    motion_mask = cv2.medianBlur(thresh, 5) #use thresh variable if cv2.threshold
    #motion_mask = cv2.GaussianBlur(thresh, (15, 15))
    # morphological operations
    motion_mask = cv2.morphologyEx(motion_mask, cv2.MORPH_OPEN, kernel, iterations=1)
    motion_mask = cv2.morphologyEx(motion_mask, cv2.MORPH_CLOSE, kernel, iterations=1)

    return motion_mask

In [4]:
sub_type = 'MOG2' # 'KNN'


if sub_type == 'MOG2':
    backSub = cv2.createBackgroundSubtractorMOG2(varThreshold=16, detectShadows=False)
else:
    backSub = cv2.createBackgroundSubtractorKNN(dist2Threshold=1000, detectShadows=False)

thresh = 500

break_out_var = False
for video in video_files:

    cap = cv2.VideoCapture(video)
    
    while(cap.isOpened()):
        
        ret, frame = cap.read()
        if ret == True:
          
            height, width, layers = frame.shape
            new_h = 360 #int(height / 2)
            new_w = 600 #int(width / 2)
            frame = cv2.resize(frame, (new_w, new_h))
            fgMask = backSub.apply(frame)
            motion_mask = get_motion_mask(fgMask, min_thresh = 50)
    
            contours, _ = cv2.findContours(motion_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_TC89_L1)
            
            for cnt in contours:
                x,y,w,h = cv2.boundingRect(cnt)
                area = w*h
                if area > thresh:
                    frame = cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), thickness = 2)
    
            cv2.imshow('normal video',frame)
            #cv2.imshow('fg_mask',motion_mask)
    
            k = cv2.waitKey(10) & 0xFF
            if k  == ord('q'):
                break
            elif k == ord('w'):
                break_out_var = True
                break
    
        else:
            break

    if (break_out_var):
        break
    
    cap.release()
    
    cv2.destroyAllWindows()